In [117]:
import geopandas as gpd 
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import h3
import json
import folium
from geojson.feature import *
import streamlit as st
import branca.colormap as cm 
import folium
from streamlit_folium import folium_static
from folium import Map, Marker, GeoJson 


gdf = gpd.read_file('../data/kontur_population_KE_20231101.gpkg')

In [118]:
def print_df_info(df, df_name):
    print(f"Data Structure for {df_name}")
    print("-"*10)
    print(f"Dimensions: {df.shape}")
    print(f"Data Types:\n{df.dtypes}")
    print(f"Missing Values:\n{df.isnull().sum()}")
    print(f"Unique observations:\n{df.nunique()}")
    print('\n')

## Merging population data and the age demographics data

In [119]:
df_demographics = pd.read_csv("./output/KEN_agesex_aggregated.csv")

In [120]:
def get_pop_total_and_sum(df):
    total_pop = df['total_population'].round().astype(int)
    sum_gender = (df['total_male'].round().astype(int) + df['total_female'].round().astype(int))

    # Count the number of rows where they match
    matching_rows = (total_pop == sum_gender).sum()
    return total_pop, sum_gender, matching_rows

def process_population_data(df):
    """
    Process population data by:
    1. Filling missing population values with sum of demographic columns
    2. Creating total male and female population columns
    
    Parameters:
    df (pandas.DataFrame): DataFrame containing population data
    
    Returns:
    pandas.DataFrame: Processed DataFrame with new columns
    """
    # Create a copy to avoid modifying the original
    processed_df = df.copy()
    
    # Get all demographic columns (excluding 'h3', 'population', and 'geometry')
    demographic_cols = [col for col in df.columns 
                       if col.startswith('population_') 
                       and col != 'population']
    
    # Fill missing population values with sum of demographic columns
    processed_df['population'] = processed_df['population'].fillna(
        processed_df[demographic_cols].sum(axis=1)
    )
    
    # Create total male population column
    male_cols = [col for col in df.columns if col.startswith('population_m')]
    processed_df['total_male'] = processed_df[male_cols].sum(axis=1).round().astype(int)
    
    # Create total female population column
    female_cols = [col for col in df.columns if col.startswith('population_f')]
    processed_df['total_female'] = processed_df[female_cols].sum(axis=1).round().astype(int)

    processed_df.rename(columns={'population': 'total_population'}, inplace=True)
    processed_df.drop('geometry', axis=1, inplace=True)

    total_pop, sum_gender, matching_rows = get_pop_total_and_sum(processed_df)

    print(f"Number of rows where total population equals sum of gender totals: {matching_rows}")

    # negative value indicates that the sum is bigger than the recorded total pop from first dataset 
    # positive value indicates that the recorded total pop from first dataset is bigger than the sum
    processed_df['recorded_pop_diff'] = total_pop - sum_gender

    #Update the total_population to capture the max number of population 
    # Do this before update to keep the original difference recorded
    # Create the condition where sum is bigger than total population
    condition = (processed_df['total_male'] + processed_df['total_female']) > processed_df['total_population']

    # Replace population where condition is True
    processed_df.loc[condition, 'total_population'] = processed_df.loc[condition, 'total_male'] + processed_df.loc[condition, 'total_female']
    total_pop, sum_gender, matching_rows = get_pop_total_and_sum(processed_df)
    print(f"Number of rows where total population equals sum of gender totals after update: {matching_rows}")

    
    return processed_df

In [121]:
gdf = gdf.to_crs(epsg=4326)
df_merged = df_demographics.merge(gdf, on='h3', how='left') 

In [ ]:
df_merged_clean = process_population_data(df_merged)
def get_h3_center(h3_index):
    # h3.h3_to_geo returns (lat, lng)
    lat, lng = h3.h3_to_geo(h3_index)
    return pd.Series({'latitude': lat, 'longitude': lng})

# Apply the function to create new columns
df_merged_clean[['latitude', 'longitude']] = df_merged_clean['h3'].apply(get_h3_center)

Number of rows where total population equals sum of gender totals: 40548
Number of rows where total population equals sum of gender totals after update: 143855


In [124]:
df_merged_clean.columns

Index(['h3', 'population_m10', 'population_f45', 'population_f50',
       'population_f40', 'population_f55', 'population_f80', 'population_m15',
       'population_f25', 'population_f30', 'population_m65', 'population_m70',
       'population_m60', 'population_m75', 'population_f20', 'population_f35',
       'population_f10', 'population_m1', 'population_m50', 'population_m45',
       'population_m0', 'population_m55', 'population_m40', 'population_m5',
       'population_f15', 'population_m80', 'population_m30', 'population_m25',
       'population_f5', 'population_f70', 'population_f65', 'population_f0',
       'population_f75', 'population_f60', 'population_f1', 'population_m35',
       'population_m20', 'total_population', 'total_male', 'total_female',
       'recorded_pop_diff', 'latitude', 'longitude'],
      dtype='object')

In [ ]:
# Basic statistics
print("Basic Statistics:")
print(f"Minimum: {df_merged_clean['recorded_pop_diff'].min()}")
print(f"Maximum: {df_merged_clean['recorded_pop_diff'].max()}")
print(f"Mean: {df_merged_clean['recorded_pop_diff'].mean():.2f}")
print(f"Median: {df_merged_clean['recorded_pop_diff'].median()}")
print(f"Mode: {df_merged_clean['recorded_pop_diff'].mode()[0]}")  # [0] because mode can return multiple values
print(f"Standard Deviation: {df_merged_clean['recorded_pop_diff'].std():.2f}")

# Get quartile information
print("\nQuartile Information:")
print(df_merged_clean['recorded_pop_diff'].describe())

# Count the frequency of each unique value
print("\nValue Counts:")
print(df_merged_clean['recorded_pop_diff'].value_counts().sort_index())

# Optional: Create a histogram to visualize the distribution
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.histplot(data=df_merged_clean, x='recorded_pop_diff', bins=30)
plt.title('Distribution of Differences between Total Population and Sum of Genders')
plt.xlabel('Difference')
plt.ylabel('Count')
plt.show()

# Optional: Add a box plot to see outliers
plt.figure(figsize=(10, 4))
sns.boxplot(x=df_merged_clean['recorded_pop_diff'])
plt.title('Box Plot of Differences')
plt.show()

## Poverty data County (doing merge with h3 population here)

In [697]:
sub_counties = gpd.read_file('../data/ken_adm_iebc_20191031_shp/ken_admbnda_adm2_iebc_20191031.shp')

In [ ]:
sub_counties[sub_counties['ADM1_EN'] == 'Mombasa']

In [ ]:
sub_counties['ADM2_EN'].value_counts()

In [126]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

# Read the shapefile
counties = gpd.read_file('../data/ken_adm_iebc_20191031_shp/ken_admbnda_adm1_iebc_20191031.shp')

In [127]:
def get_county_from_coordinates(df, counties):
    geometry = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]
    h3_gdf = gpd.GeoDataFrame(df, geometry=geometry, crs='EPSG:4326')

    # 2. Perform spatial join
    # This will add all columns from counties to your H3 data
    result = gpd.sjoin(h3_gdf, counties, how='left', predicate='within')

    # 3. If you only want to keep the county name (assuming it's in 'ADM1_EN')
    # Keep only the original columns plus the county name
    original_columns = list(df.columns)
    df['county'] = result['ADM1_EN']

    # 4. Some hexagons are left unmatched so perform other operations to match them with a county
    unmatched_hexagons = df[df['county'].isna()].copy()
    geometry = [Point(xy) for xy in zip(unmatched_hexagons['longitude'], unmatched_hexagons['latitude'])]
    h3_gdf = gpd.GeoDataFrame(unmatched_hexagons, geometry=geometry, crs='EPSG:4326')
    intersect_match = gpd.sjoin(h3_gdf, counties, how='left', predicate='intersects')


    def assign_nearest_county(point, counties_gdf):
        # Calculate distance to all counties
        distances = counties_gdf.geometry.distance(point.geometry)
        # Get the county with minimum distance
        nearest_county = counties_gdf.iloc[distances.argmin()]
        return nearest_county['ADM1_EN']

    # For any hexagons still unmatched after intersects, find nearest county
    still_unmatched = intersect_match[intersect_match['ADM1_EN'].isna()]
    if len(still_unmatched) > 0:
        still_unmatched['nearest_county'] = still_unmatched.apply(
            lambda x: assign_nearest_county(x, counties), axis=1
        )


    df.loc[still_unmatched.index, 'county'] = still_unmatched['nearest_county']
    return df

In [ ]:
h3_df_county = get_county_from_coordinates(df_merged_clean.copy(), counties.copy())

/var/folders/b7/2dqsctv50sbgk3bg9gpz453c0000gn/T/ipykernel_22851/1926180795.py:23: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  distances = counties_gdf.geometry.distance(point.geometry)


In [ ]:
poverty_df = pd.read_csv('../data/KEN_MPI_COUNTY_PROCESSED.csv')
poverty_df = poverty_df.rename(columns={'Subnational region': 'county'})
poverty_df.drop(['Country', 'ISO country numeric code', 'ISO country code','World region'], axis=1, inplace=True)
h3_with_poverty = h3_df_county.merge(poverty_df, 
                             on='county', 
                             how='left')

## Financial Services Polygons

In [494]:
financial_services = gpd.read_file('../data/ken_financial_services_polygons/ken_financial_services_polygons.gpkg')
financial_services_points = gpd.read_file('../data/ken_financial_services_points/ken_financial_services_points.gpkg')

In [495]:
financial_services.drop(['name:en', 'operator', 'network', 'addr:full', 'addr:city', 'source', 'name:sw'], axis=1, inplace=True)
financial_services_points.drop(['name:en', 'operator', 'network', 'addr:full', 'addr:city', 'source', 'name:sw'], axis=1, inplace=True)

In [501]:
print_df_info(financial_services_points, 'financial_services_points')

Data Structure for financial_services_points
----------
Dimensions: (1720, 7)
Data Types:
name           object
amenity        object
osm_id          int64
osm_type       object
geometry     geometry
longitude     float64
latitude      float64
dtype: object
Missing Values:
name         227
amenity        0
osm_id         0
osm_type       0
geometry       0
longitude      0
latitude       0
dtype: int64
Unique observations:
name          505
amenity         7
osm_id       1720
osm_type        1
geometry     1719
longitude    1717
latitude     1713
dtype: int64




In [ ]:
financial_services_points['amenity'].value_counts()

In [477]:
def extract_coordinates_fin_services(gdf):
    """
    Extract longitude and latitude from geometry column of a GeoDataFrame.
    
    Parameters:
    gdf (GeoDataFrame): Input GeoDataFrame with geometry column
    
    Returns:
    GeoDataFrame: Original GeoDataFrame with new longitude and latitude columns
    """
    # Create a copy to avoid modifying the original
    gdf = gdf.copy()
    
    # Extract coordinates
    gdf['longitude'] = gdf.geometry.centroid.x
    gdf['latitude'] = gdf.geometry.centroid.y
    
    # Verify the results
    print("Number of points processed:", len(gdf))
    print("Number of missing coordinates:", gdf[['longitude', 'latitude']].isna().any(axis=1).sum())
    
    return gdf

In [496]:
financial_services_points = extract_coordinates_fin_services(financial_services_points)

Number of points processed: 1720
Number of missing coordinates: 0


/var/folders/b7/2dqsctv50sbgk3bg9gpz453c0000gn/T/ipykernel_22851/649738154.py:15: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['longitude'] = gdf.geometry.centroid.x
/var/folders/b7/2dqsctv50sbgk3bg9gpz453c0000gn/T/ipykernel_22851/649738154.py:16: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['latitude'] = gdf.geometry.centroid.y


In [ ]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time

def lookup_places_osm(df):
    """
    Look up place names using OpenStreetMap's Nominatim for coordinates with missing names.
    """
    # Initialize geocoder
    geolocator = Nominatim(user_agent="ken_financial_services_points_app")
    
    df = df.copy()
    count = 0
    count_null_names_before = df['name'].isna().sum()
    print('[INFO] Attempting to fill out the null names in the financial services...')
    for idx in df[df['name'].isna()].index:
        lat = df.loc[idx, 'latitude']
        lng = df.loc[idx, 'longitude']
        
        try:
            # Reverse geocode the coordinates
            location = geolocator.reverse(f"{lat}, {lng}", exactly_one=True)
            
            if location:
                # Extract the name from address data
                address = location.raw.get('address', {})
                print(f"Address extracted = {address}")
                if 'amenity' in address:
                    print(f'[INFO] Amenity Name found!\n')
                    df.loc[idx, 'name'] = address['amenity']
            
            # Sleep to respect rate limits
            time.sleep(1)  # OpenStreetMap requires slower rates
            
            count += 1
            if count % 5 == 0:
                print(f"Processed {count} locations")
                
        except Exception as e:
            print(f"Error processing coordinates ({lat}, {lng}): {str(e)}")
            continue
    
    count_null_names_after = df['name'].isna().sum()
    print(f"Null values in the name column before processing = {count_null_names_before}")
    print(f"Null values in the name column after processing = {count_null_names_after}\n")
    return df

In [527]:
financial_amenities = ['bank', 'atm', 'mobile_money_agent', 'bureau_de_change', 'money_transfer', 'sacco']

# Filter rows where 'amenity' is in the list
financial_services_points_no_office = financial_services_points[financial_services_points['amenity'].isin(financial_amenities)]

In [528]:
financial_services_points_no_office['name'].nunique()

359

In [ ]:
financial_services_points_no_office['name'].value_counts()

In [ ]:
financial_services_points = lookup_places_osm_fin(financial_services_points)

In [413]:
financial_services_with_coords = extract_coordinates_fin_services(financial_services)
# Banks or ATMS
financial_services_with_coords.loc[financial_services_with_coords['osm_id'] == 510542492, 'name'] = 'KCB Bank'
financial_services_with_coords.loc[financial_services_with_coords['osm_id'] == 927298614, 'name'] = 'Sidian Bank'
financial_services_with_coords.loc[financial_services_with_coords['osm_id'] == 694282218, 'name'] = 'National Bank'
financial_services_with_coords.loc[financial_services_with_coords['osm_id'] == 206638774, 'name'] = 'KCB Bank'
financial_services_with_coords.loc[financial_services_with_coords['osm_id'] == 488367319, 'name'] = 'Rafiki Microfinance Bank'
financial_services_with_coords.loc[financial_services_with_coords['osm_id'] == 488598405, 'name'] = 'Tower Sacco Society Limited'
financial_services_with_coords.loc[financial_services_with_coords['osm_id'] == 1092605098, 'name'] = 'Equity Bank'
financial_services_with_coords.loc[financial_services_with_coords['osm_id'] == 927298612, 'name'] = 'Equity Bank'
financial_services_with_coords.loc[financial_services_with_coords['osm_id'] == 927298613, 'name'] = 'Equity Bank'
financial_services_with_coords.loc[financial_services_with_coords['osm_id'] == 1156337156, 'name'] = 'Trans National Bank'
financial_services_with_coords.loc[financial_services_with_coords['osm_id'] == 331976738, 'name'] = 'Kenya Commercial Bank'
financial_services_with_coords.loc[financial_services_with_coords['osm_id'] == 988844673, 'name'] = 'Sanrock mpesa'

# Post offices
financial_services_with_coords.loc[financial_services_with_coords['osm_id'] == 386760599, 'name'] = 'Fargo Courier'
financial_services_with_coords.loc[financial_services_with_coords['osm_id'] == 322160157, 'name'] = 'POSTA'
financial_services_with_coords.loc[financial_services_with_coords['osm_id'] == 313516167, 'name'] = 'POSTA'

# Set name to unknown if google maps has no data about the institution when looking up the coordinates
financial_services_with_coords.loc[financial_services_with_coords['osm_id'] == 580833129, 'name'] = 'Unknown'

Number of points processed: 131
Number of missing coordinates: 0


/var/folders/b7/2dqsctv50sbgk3bg9gpz453c0000gn/T/ipykernel_22851/649738154.py:15: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['longitude'] = gdf.geometry.centroid.x
/var/folders/b7/2dqsctv50sbgk3bg9gpz453c0000gn/T/ipykernel_22851/649738154.py:16: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['latitude'] = gdf.geometry.centroid.y


In [492]:
# Function to standardize bank names
def standardize_bank_name(df, pattern, standard_name):
    """
    Standardize bank names based on a pattern.
    
    Parameters:
    df: DataFrame containing bank names
    pattern: String pattern to match (e.g., 'KCB')
    standard_name: Name to replace matches with (e.g., 'KCB Bank')
    """
    if (type(pattern) == str):
        # Show current variations
        print(f"Current variations of names containing '{pattern}':")
        print(df[df['name'].str.contains(pattern, case=False, na=False)]['name'].unique())
        
        # Standardize the name
        df.loc[df['name'].str.contains(pattern, case=False, na=False), 'name'] = standard_name
        
        # Verify the change
        print(f"\nAfter standardization - all matched entries now show as '{standard_name}'")
        print(df[df['name'].str.contains(pattern, case=False, na=False)]['name'].unique())
    
    else:
        print("Processing multiple patterns at once")
        for pat in pattern:
            # Show current variations
            print(f"Current variations of names containing '{pat}':")
            print(df[df['name'].str.contains(pat, case=False, na=False)]['name'].unique())
            
            # Standardize the name
            df.loc[df['name'].str.contains(pat, case=False, na=False), 'name'] = standard_name
            
            # Verify the change
            print(f"\nAfter standardization - all matched entries now show as '{standard_name}'")
            print(df[df['name'].str.contains(pat, case=False, na=False)]['name'].unique())

    
    return df

In [ ]:
# Renaming banks since a lot of them represent the same bank 
financial_services_points.loc[financial_services_points['name'] == 'K C B', 'name'] = 'KCB Bank'
financial_services_points = standardize_bank_name(financial_services_points, 'KCB', 'KCB Bank')
financial_services_points = standardize_bank_name(financial_services_points, 'Kenya Commercial Bank', 'KCB Bank')

financial_services_points = standardize_bank_name(financial_services_points, 'Stanbic', 'Stanbic Bank')
financial_services_points = standardize_bank_name(financial_services_points, 'Family Bank', 'Family Bank')

financial_services_points = standardize_bank_name(financial_services_points, 'Commercial Bank of', 'Commercial Bank of Africa')
financial_services_points = standardize_bank_name(financial_services_points, 'Bank of Africa', 'Commercial Bank of Africa')
financial_services_points = standardize_bank_name(financial_services_points, 'NIC Bank Limited', 'Commercial Bank of Africa')

financial_services_points = standardize_bank_name(financial_services_points, 'CBK', 'Cooperative Bank of Kenya') 
financial_services_points = standardize_bank_name(financial_services_points, 'Cooperative Bank', 'Cooperative Bank of Kenya') 
financial_services_points = standardize_bank_name(financial_services_points, 'Co-operative Bank', 'Cooperative Bank of Kenya')

financial_services_points = standardize_bank_name(financial_services_points, 'I&M', 'I&M Bank')

financial_services_points = standardize_bank_name(financial_services_points, 'Post Bank', 'Kenya Post Office Savings Bank')
financial_services_points = standardize_bank_name(financial_services_points, 'Postbank', 'Kenya Post Office Savings Bank')

financial_services_points = standardize_bank_name(financial_services_points, 'Chase', 'Chase Bank')
financial_services_points = standardize_bank_name(financial_services_points, 'ChaseBank', 'Chase Bank')

financial_services_points = standardize_bank_name(financial_services_points, 'KWFT', 'Kenya Women Finance Trust')
financial_services_points = standardize_bank_name(financial_services_points, 'Kenya Women Finance Trust', 'Kenya Women Finance Trust')

financial_services_points = standardize_bank_name(financial_services_points, 'Trans National Bank', 'Access Bank Kenya')

financial_services_points = standardize_bank_name(financial_services_points, 'National Bank', 'National Bank of Kenya')

financial_services_points = standardize_bank_name(financial_services_points, 'Equity', 'Equity Bank')

financial_services_points = standardize_bank_name(financial_services_points, 'Barclays', 'Barclays Bank')
financial_services_points = standardize_bank_name(financial_services_points, 'Barcalys', 'Barclays Bank')


financial_services_points = standardize_bank_name(financial_services_points, 'Co-Op Bank', 'Co-operative Bank of Kenya')
financial_services_points = standardize_bank_name(financial_services_points, 'Co-op', 'Co-operative Bank of Kenya')
financial_services_points = standardize_bank_name(financial_services_points, 'Co-', 'Co-operative Bank of Kenya')
financial_services_points = standardize_bank_name(financial_services_points, ['Co operative', 'Co -', 'Co Op Bank', 'Co Op', 'Coop Bank'], 'Co-operative Bank of Kenya')

financial_services_points = standardize_bank_name(financial_services_points, 'Ecobank', 'EcoBank')

financial_services_points = standardize_bank_name(financial_services_points, 'Absa', 'Absa Bank')


financial_services_points = standardize_bank_name(financial_services_points, 'DAIMOND', 'Diamond Trust Bank Group')
financial_services_points = standardize_bank_name(financial_services_points, 'DTB', 'Diamond Trust Bank Group')
financial_services_points = standardize_bank_name(financial_services_points, 'Diamond Trust Bank', 'Diamond Trust Bank Group')

financial_services_points = standardize_bank_name(financial_services_points, 'Sacco', 'Savings and Credit Cooperative Organization')

financial_services_points = standardize_bank_name(financial_services_points, 'M-Pesa', 'M-Pesa Point')
financial_services_points = standardize_bank_name(financial_services_points, 'mpesa', 'M-Pesa Point')
financial_services_points = standardize_bank_name(financial_services_points, ['Pesa', 'psea'], 'M-Pesa Point')


financial_services_points = standardize_bank_name(financial_services_points, ['Kenya Women Finance', 'Kenya Woman Finance'], 'Kenya Women Finance Trust')
financial_services_points = standardize_bank_name(financial_services_points, ['Kenya Women Micro', 'Kenya Woman Micro'], 'Kenya Women Microfinance Bank')

# Rename all variations to "Posta" (post office)
financial_services_points.loc[financial_services_points['name'].str.contains('Posta', case=False, na=False), 'name'] = 'POSTA'

In [ ]:
# Renaming banks since a lot of them represent the same bank 
financial_services_with_coords.loc[financial_services_with_coords['name'] == 'K C B', 'name'] = 'KCB Bank'
financial_services_with_coords = standardize_bank_name(financial_services_with_coords, 'KCB', 'KCB Bank')
financial_services_with_coords = standardize_bank_name(financial_services_with_coords, 'Kenya Commercial Bank', 'KCB Bank')

financial_services_with_coords = standardize_bank_name(financial_services_with_coords, 'Stanbic', 'Stanbic Bank')
financial_services_with_coords = standardize_bank_name(financial_services_with_coords, 'Family Bank', 'Family Bank')

financial_services_with_coords = standardize_bank_name(financial_services_with_coords, 'Commercial Bank of', 'Commercial Bank of Africa')
financial_services_with_coords = standardize_bank_name(financial_services_with_coords, 'Bank of Africa', 'Commercial Bank of Africa')
financial_services_with_coords = standardize_bank_name(financial_services_with_coords, 'NIC Bank Limited', 'Commercial Bank of Africa')

financial_services_with_coords = standardize_bank_name(financial_services_with_coords, 'CBK', 'Cooperative Bank of Kenya') 
financial_services_with_coords = standardize_bank_name(financial_services_with_coords, 'Cooperative Bank', 'Cooperative Bank of Kenya') 
financial_services_with_coords = standardize_bank_name(financial_services_with_coords, 'Co-operative Bank', 'Cooperative Bank of Kenya')

financial_services_with_coords = standardize_bank_name(financial_services_with_coords, 'I&M', 'I&M Bank')

financial_services_with_coords = standardize_bank_name(financial_services_with_coords, 'Post Bank', 'Kenya Post Office Savings Bank')

financial_services_with_coords = standardize_bank_name(financial_services_with_coords, 'KWFT', 'Kenya Women Finance Trust')
financial_services_with_coords = standardize_bank_name(financial_services_with_coords, 'Kenya Women Finance Trust', 'Kenya Women Finance Trust')

financial_services_with_coords = standardize_bank_name(financial_services_with_coords, 'Trans National Bank', 'Access Bank Kenya')

financial_services_with_coords = standardize_bank_name(financial_services_with_coords, 'National Bank', 'National Bank of Kenya')

financial_services_with_coords = standardize_bank_name(financial_services_with_coords, 'Equity', 'Equity Bank')

financial_services_with_coords = standardize_bank_name(financial_services_with_coords, 'Barclays', 'Barclays Bank')

financial_services_with_coords = standardize_bank_name(financial_services_with_coords, 'DAIMOND', 'Diamond Trust Bank Group')
financial_services_with_coords = standardize_bank_name(financial_services_with_coords, 'DTB', 'Diamond Trust Bank Group')
financial_services_with_coords = standardize_bank_name(financial_services_with_coords, 'Diamond Trust Bank', 'Diamond Trust Bank Group')

financial_services_with_coords = standardize_bank_name(financial_services_with_coords, 'Sacco', 'Savings and Credit Cooperative Organization')

financial_services_with_coords = standardize_bank_name(financial_services_with_coords, 'M-Pesa', 'M-Pesa Point')
financial_services_with_coords = standardize_bank_name(financial_services_with_coords, 'mpesa', 'M-Pesa Point')

# Rename all variations to "Posta" (post office)
financial_services_with_coords.loc[financial_services_with_coords['name'].str.contains('Posta', case=False, na=False), 'name'] = 'POSTA'

In [185]:
pd.set_option('display.max_rows', None)

## Health facilities Polygons

In [389]:
health_facilities = gpd.read_file('../data/ken_health_facilities_polygons/ken_health_facilities_polygons.gpkg')

In [421]:
health_facilities_points = gpd.read_file('../data/ken_health_facilities_points/ken_health_facilities_points.gpkg')

In [ ]:
health_facilities_points['amenity'].value_counts()

In [422]:
print_df_info(health_facilities_points, 'health_facilities_points')

Data Structure for health_facilities_points
----------
Dimensions: (1761, 15)
Data Types:
name                       object
name:en                    object
amenity                    object
building                   object
healthcare                 object
healthcare:speciality      object
operator:type              object
capacity:persons           object
addr:full                  object
addr:city                  object
source                     object
name:sw                    object
osm_id                      int64
osm_type                   object
geometry                 geometry
dtype: object
Missing Values:
name                       74
name:en                  1748
amenity                    37
building                 1758
healthcare               1124
healthcare:speciality    1667
operator:type            1155
capacity:persons         1761
addr:full                1760
addr:city                1610
source                   1612
name:sw                  1761
osm_id    

In [390]:
health_facilities.drop(['name:en', 'operator:type', 'capacity:persons', 'addr:full', 'addr:city', 'source', 'name:sw', 'healthcare:speciality'], axis=1, inplace=True)

In [330]:
health_facilities['healthcare'].value_counts()

healthcare
hospital            518
clinic               23
yes                   7
pharmacy              5
laboratory            2
centre                2
doctor                1
level_3_hospital      1
optometrist           1
Name: count, dtype: int64

In [561]:
def extract_coordinates_from_geometry(gdf):
    """
    Extract longitude and latitude from geometry column of a GeoDataFrame.
    
    Parameters:
    gdf (GeoDataFrame): Input GeoDataFrame with geometry column
    
    Returns:
    GeoDataFrame: Original GeoDataFrame with new longitude and latitude columns
    """
    # Create a copy to avoid modifying the original
    gdf = gdf.copy()
    
    # Extract coordinates
    gdf['longitude'] = gdf.geometry.centroid.x
    gdf['latitude'] = gdf.geometry.centroid.y
    
    # Verify the results
    print("Number of points processed:", len(gdf))
    print("Number of missing coordinates:", gdf[['longitude', 'latitude']].isna().any(axis=1).sum())
    
    return gdf

In [ ]:
health_facilities = extract_coordinates_from_geometry(health_facilities)
health_facilities.loc[health_facilities['osm_id'] == 792488917, 'amenity'] = 'hospital'
health_facilities.loc[health_facilities['osm_id'] == 1166210971, 'amenity'] = 'clinic'
health_facilities.loc[health_facilities['osm_id'] == 1126947216, 'amenity'] = 'clinic'
health_facilities.loc[health_facilities['osm_id'] == 449705379, 'amenity'] = 'clinic'
health_facilities.loc[health_facilities['osm_id'] == 1064641276, 'amenity'] = 'clinic'


# Fill out the amenities for entries corresponding to hospitals 
health_facilities.loc[health_facilities['osm_id'] == 1167183485, 'amenity'] = 'hospital'
health_facilities.loc[health_facilities['osm_id'] == 691508967, 'amenity'] = 'hospital'
health_facilities.loc[health_facilities['osm_id'] == 656482162, 'amenity'] = 'hospital'
health_facilities.loc[health_facilities['osm_id'] == 364992933, 'amenity'] = 'hospital'


# Checked website for this one and it says clinic
health_facilities.loc[health_facilities['osm_id'] == 798278750, 'amenity'] = 'clinic'
health_facilities.loc[health_facilities['osm_id'] == 1064641270, 'amenity'] = 'clinic'

# Checked google maps using coordinates 
health_facilities.loc[health_facilities['osm_id'] == 1170419534, 'amenity'] = 'pharmacy'
health_facilities.loc[health_facilities['osm_id'] == 1170410092, 'amenity'] = 'pharmacy'
health_facilities.loc[health_facilities['osm_id'] == 1316676525, 'amenity'] = 'unknown'


Number of points processed: 684
Number of missing coordinates: 0


/var/folders/b7/2dqsctv50sbgk3bg9gpz453c0000gn/T/ipykernel_22851/2997271296.py:15: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['longitude'] = gdf.geometry.centroid.x
/var/folders/b7/2dqsctv50sbgk3bg9gpz453c0000gn/T/ipykernel_22851/2997271296.py:16: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['latitude'] = gdf.geometry.centroid.y


In [ ]:
missing_amenities = health_facilities[health_facilities['amenity'].isna()].copy()

# Create a format that's easy to copy-paste into Google Maps
missing_amenities['google_maps_format'] = missing_amenities['latitude'].astype(str) + ', ' + missing_amenities['longitude'].astype(str)

In [568]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time

def lookup_places_osm(df):
    """
    Look up place names using OpenStreetMap's Nominatim for coordinates with missing names.
    """
    # Initialize geocoder
    geolocator = Nominatim(user_agent="my_health_facilities_app")
    
    df = df.copy()
    count = 0
    count_null_names_before = df['name'].isna().sum()
    for idx in df[df['name'].isna()].index:
        lat = df.loc[idx, 'latitude']
        lng = df.loc[idx, 'longitude']
        
        try:
            # Reverse geocode the coordinates
            location = geolocator.reverse(f"{lat}, {lng}", exactly_one=True)
            
            if location:
                # Extract the name from address data
                address = location.raw.get('address', {})
                print(f"Address extracted = {address}")
                if 'amenity' in address:
                    df.loc[idx, 'name'] = address['amenity']
                elif 'healthcare' in address:
                    df.loc[idx, 'name'] = address['healthcare']
            
            # Sleep to respect rate limits
            time.sleep(1)  # OpenStreetMap requires slower rates
            
            count += 1
            if count % 5 == 0:
                print(f"Processed {count} locations")
                
        except Exception as e:
            print(f"Error processing coordinates ({lat}, {lng}): {str(e)}")
            continue

    count_null_names_after = df['name'].isna().sum()
    print(f"Null values in the name column before processing = {count_null_names_before}")
    print(f"Null values in the name column after processing = {count_null_names_after}")
    return df

In [ ]:
# Use the OSM version (no API key needed)
health_facilities_with_names = lookup_places_osm(health_facilities)
health_facilities_with_names.loc[health_facilities_with_names['name'] == 'University of Eldoret' , 'name'] = 'Chepkoilel Sisiwa Dispensary'
health_facilities_with_names.loc[health_facilities_with_names['name'] == 'Word Temple Church' , 'name'] = 'Grace Medical Clinic'
health_facilities_with_names['name'] = health_facilities_with_names['name'].fillna(health_facilities_with_names['amenity'])

## Railways Data

In [ ]:
railways_lines = gpd.read_file('../data/ken_railways_lines/ken_railways_lines.gpkg')

In [380]:
railways_lines.drop(['name:en', 'ele', 'operator:type', 'layer', 'addr:full', 'addr:city', 'source', 'name:sw'], axis=1, inplace=True)

In [387]:
railways_lines['name'] = railways_lines['name'].fillna('Name Not Listed')

In [ ]:
print_df_info(railways_lines, 'railways_lines')

# Points of interest

In [632]:
points_of_interest_points = gpd.read_file('../data/ken_points_of_interest_points/ken_points_of_interest_points.gpkg')

## Healthcare facilities

In [633]:
health_amenities = ['pharmacy', 'clinic', 'hospital', 'veterinary', 'dispensary', 'dentist', '*', 'nursing_home', 'doctors', 'medical transport service','health_post', 'health_center','health_centre','pharmaccy','traditional health centre','veterinary_pharmacy', 'healthcare','clinic;hospital','health']
health_facilities_points = points_of_interest_points[points_of_interest_points['amenity'].isin(health_amenities)]

# Drop the rows pertaining to health_facilities
points_of_interest_points = points_of_interest_points[~points_of_interest_points['amenity'].isin(health_amenities)]

In [634]:
health_facilities_points.loc[health_facilities_points['amenity'] == 'pharmaccy', 'amenity'] = 'pharmacy'
health_facilities_points.loc[health_facilities_points['amenity'] == 'health_centre', 'amenity'] = 'health_center'
health_facilities_points.loc[health_facilities_points['amenity'] == 'health', 'amenity'] = 'health_center'
health_facilities_points.loc[health_facilities_points['amenity'] == 'healthcare', 'amenity'] = 'health_center'
health_facilities_points.loc[health_facilities_points['amenity'] == 'clinic;hospital', 'amenity'] = 'clinic'

In [ ]:
health_facilities_points.drop(['name:en', 'man_made', 'shop', 'tourism',
       'opening_hours', 'beds', 'rooms', 'addr:full', 'addr:housenumber',
       'addr:street', 'addr:city', 'source', 'name:sw'], axis = 1, inplace=True)

health_facilities_points = extract_coordinates_from_geometry(health_facilities_points)
health_facilities_points = lookup_places_osm(health_facilities_points)

Number of points processed: 1824
Number of missing coordinates: 0


/var/folders/b7/2dqsctv50sbgk3bg9gpz453c0000gn/T/ipykernel_22851/3776026523.py:15: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['longitude'] = gdf.geometry.centroid.x
/var/folders/b7/2dqsctv50sbgk3bg9gpz453c0000gn/T/ipykernel_22851/3776026523.py:16: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['latitude'] = gdf.geometry.centroid.y


In [614]:
# Create a format that's easy to copy-paste into Google Maps
health_facilities_points['google_maps_format'] = health_facilities_points['latitude'].astype(str) + ', ' + health_facilities_points['longitude'].astype(str)

In [615]:
len(points_of_interest_points)

75119

In [636]:
# Get rid of the financial services data from the points of interest data
financial_amenities = ['bank', 'atm', 'mobile_money_agent', 'post_office', 'bureau_de_change', 'money_transfer', 'sacco']

# Filter rows where 'amenity' is in the list
points_of_interest_points = points_of_interest_points[~points_of_interest_points['amenity'].isin(financial_amenities)]

## Education Facilities

In [637]:
educational_amenities = ['school', 'driving_school', 'university', 'library', 'college', 'kindergarten']

educational_facilities_points = points_of_interest_points[points_of_interest_points['amenity'].isin(educational_amenities)]


points_of_interest_points = points_of_interest_points[~points_of_interest_points['amenity'].isin(educational_amenities)]


In [ ]:
educational_facilities_points.drop(['name:en', 'man_made', 'shop', 'tourism',
       'opening_hours', 'beds', 'rooms', 'addr:full', 'addr:housenumber',
       'addr:street', 'addr:city', 'source', 'name:sw'], axis = 1, inplace=True)

educational_facilities_points = extract_coordinates_from_geometry(educational_facilities_points)
educational_facilities_points = lookup_places_osm(educational_facilities_points)

Number of points processed: 5196
Number of missing coordinates: 0


/var/folders/b7/2dqsctv50sbgk3bg9gpz453c0000gn/T/ipykernel_22851/3776026523.py:15: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['longitude'] = gdf.geometry.centroid.x
/var/folders/b7/2dqsctv50sbgk3bg9gpz453c0000gn/T/ipykernel_22851/3776026523.py:16: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['latitude'] = gdf.geometry.centroid.y


In [ ]:
print_df_info(educational_facilities_points, 'educational_facilities_points')

In [639]:
wrong_amenities = ['fixme', 'no', 'Mission']
amenity_value_counts = points_of_interest_points['amenity'].value_counts()
amenity_values_to_drop = amenity_value_counts[amenity_value_counts == 1].index

points_of_interest_points = points_of_interest_points[~points_of_interest_points['amenity'].isin(wrong_amenities)]
points_of_interest_points = points_of_interest_points[~points_of_interest_points['amenity'].isin(amenity_values_to_drop)]

## Transportation

In [ ]:
transportation_amenities = ['bus_station', 'taxi', 'ferry_terminal', 'bus_stop', 'train_station', 'parking', 'bicycle_parking', 'parking_space', 'motorcycle_parking', 'car_rental', 'car_sharing', 'vehicle_inspection', 'bridge', 'ferry_terminal', 'taxi_stand', 'airport', 'harbor', 'railway_station', 'fuel', 'charging_station']

transportation_points = points_of_interest_points[points_of_interest_points['amenity'].isin(transportation_amenities)]



points_of_interest_points = points_of_interest_points[~points_of_interest_points['amenity'].isin(transportation_amenities)]

In [ ]:
transportation_points.drop(['name:en', 'man_made', 'shop', 'tourism',
       'opening_hours', 'beds', 'rooms', 'addr:full', 'addr:housenumber',
       'addr:street', 'addr:city', 'source', 'name:sw'], axis = 1, inplace=True)

transportation_points = extract_coordinates_from_geometry(transportation_points)

## Public Amenities & Utilities

In [ ]:
public_amenities = ['toilets', 'shower', 'drinking_water', 'fountain', 'waste_disposal', 'waste_transfer_station', 'waste_basket', 'recycling', 'street_lamp', 'telephone', 'clock',  'pond', 'sanitary_dump_station']

public_amenities_points = points_of_interest_points[points_of_interest_points['amenity'].isin(public_amenities)]

points_of_interest_points = points_of_interest_points[~points_of_interest_points['amenity'].isin(public_amenities)]

In [684]:
public_amenities_points['amenity'].value_counts()

amenity
toilets            15662
drinking_water      3081
waste_disposal       487
waste_basket          62
shower                60
recycling             44
vending_machine       16
clock                 15
fountain              14
street_lamp           11
telephone              3
Name: count, dtype: int64

In [ ]:
public_amenities_points.drop(['name', 'name:en', 'man_made', 'shop', 'tourism',
       'opening_hours', 'beds', 'rooms', 'addr:full', 'addr:housenumber',
       'addr:street', 'addr:city', 'source', 'name:sw'], axis = 1, inplace=True)

public_amenities_points = extract_coordinates_from_geometry(public_amenities_points)

## Environmental & Agricultural

In [ ]:
agr_env_amenities = [
    'grinding_mill', 'compost_site','water_or_irrigation', 'water_point', 'watering_place',
    'agriculture', 'green_house', 'gardens', 'farming', 'cattle_breeding', 'pump_house', 'ranger_station', 'animal_shelter'
]

agr_env_points = points_of_interest_points[points_of_interest_points['amenity'].isin(agr_env_amenities)]


points_of_interest_points = points_of_interest_points[~points_of_interest_points['amenity'].isin(agr_env_amenities)]

In [655]:
agr_env_points.drop(['name', 'name:en', 'man_made', 'shop', 'tourism',
       'opening_hours', 'beds', 'rooms', 'addr:full', 'addr:housenumber',
       'addr:street', 'addr:city', 'source', 'name:sw'], axis = 1, inplace=True)

agr_env_points = extract_coordinates_from_geometry(agr_env_points)

Number of points processed: 5773
Number of missing coordinates: 0


/var/folders/b7/2dqsctv50sbgk3bg9gpz453c0000gn/T/ipykernel_22851/3776026523.py:15: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['longitude'] = gdf.geometry.centroid.x
/var/folders/b7/2dqsctv50sbgk3bg9gpz453c0000gn/T/ipykernel_22851/3776026523.py:16: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['latitude'] = gdf.geometry.centroid.y


## Food & Beverage

In [657]:
food_beverage_amenities = [
    'restaurant', 'cafe', 'bar', 'pub', 'fast_food', 'food_court', 'ice_cream', 'restaurant;bar', 'restaurant,bar', 'bbq', 'beer_garden', 'catering_service', 'food_outlet'
]

food_beverage_points = points_of_interest_points[points_of_interest_points['amenity'].isin(food_beverage_amenities)]
food_beverage_points.loc[food_beverage_points['amenity'] == 'pub', 'amenity'] = 'bar'
food_beverage_points.loc[food_beverage_points['amenity'] == 'beer_garden', 'amenity'] = 'bar'
food_beverage_points.loc[food_beverage_points['amenity'] == 'restaurant;bar', 'amenity'] = 'restaurant'
food_beverage_points.loc[food_beverage_points['amenity'] == 'restaurant;bar', 'amenity'] = 'restaurant'
food_beverage_points.loc[food_beverage_points['amenity'] == 'bbq', 'amenity'] = 'restaurant'

points_of_interest_points = points_of_interest_points[~points_of_interest_points['amenity'].isin(food_beverage_amenities)]

In [659]:
food_beverage_points.drop(['name:en', 'man_made', 'shop', 'tourism',
       'opening_hours', 'beds', 'rooms', 'addr:full', 'addr:housenumber',
       'addr:street', 'addr:city', 'source', 'name:sw'], axis = 1, inplace=True)

food_beverage_points = extract_coordinates_from_geometry(food_beverage_points)

Number of points processed: 3491
Number of missing coordinates: 0


/var/folders/b7/2dqsctv50sbgk3bg9gpz453c0000gn/T/ipykernel_22851/3776026523.py:15: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['longitude'] = gdf.geometry.centroid.x
/var/folders/b7/2dqsctv50sbgk3bg9gpz453c0000gn/T/ipykernel_22851/3776026523.py:16: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['latitude'] = gdf.geometry.centroid.y


## Public Services

In [ ]:
public_services_amenities = [
    'police', 'fire_station', 'ambulance_station', 'courthouse', 'townhall', 'public_building', 'public_facility', 'social_centre', 'community_centre', 'social_facility', 'civic_service', 'prison', 'emergency_service', 'rescue_service'
]

public_services_points = points_of_interest_points[points_of_interest_points['amenity'].isin(public_services_amenities)]
public_services_points.loc[public_services_points['amenity'] == 'public_facility', 'amenity'] = 'public_building'
public_services_points.loc[public_services_points['amenity'] == 'social_centre', 'amenity'] = 'social_facility'

points_of_interest_points = points_of_interest_points[~points_of_interest_points['amenity'].isin(public_services_amenities)]

In [ ]:
public_services_points.drop(['name:en', 'man_made', 'shop', 'tourism',
       'opening_hours', 'beds', 'rooms', 'addr:full', 'addr:housenumber',
       'addr:street', 'addr:city', 'source', 'name:sw'], axis = 1, inplace=True)

public_services_points = extract_coordinates_from_geometry(public_services_points)

In [ ]:
public_services_points['amenity'].value_counts()

## Leisure & Entertainment

In [ ]:
leisure_amenities = [
    'cinema', 'theatre', 'nightclub', 'club', 'gaming', 'arts_centre', 'exhibition_centre', 'leisure', 'music_venue', 'dance_club', 'gambling', 'sports_club', 'events_venue', 'studio', 'casino', 'internet_cafe'
]

leisure_points = points_of_interest_points[points_of_interest_points['amenity'].isin(leisure_amenities)]

points_of_interest_points = points_of_interest_points[~points_of_interest_points['amenity'].isin(leisure_amenities)]

In [671]:
leisure_points.drop(['name:en', 'man_made', 'shop', 'tourism',
       'opening_hours', 'beds', 'rooms', 'addr:full', 'addr:housenumber',
       'addr:street', 'addr:city', 'source', 'name:sw'], axis = 1, inplace=True)

leisure_points = extract_coordinates_from_geometry(leisure_points)

Number of points processed: 232
Number of missing coordinates: 0


/var/folders/b7/2dqsctv50sbgk3bg9gpz453c0000gn/T/ipykernel_22851/3776026523.py:15: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['longitude'] = gdf.geometry.centroid.x
/var/folders/b7/2dqsctv50sbgk3bg9gpz453c0000gn/T/ipykernel_22851/3776026523.py:16: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['latitude'] = gdf.geometry.centroid.y


In [673]:
leisure_points['amenity'].value_counts()

amenity
cinema               93
nightclub            59
studio               29
theatre              29
arts_centre          11
events_venue          7
exhibition_centre     2
gambling              2
Name: count, dtype: int64

## Shopping & Retail

In [ ]:
shopping_amenities = [
    'marketplace', 'shop', 'supermarket', 'butcher', 'butchery', 'shopping_mall', 'vending_machine', 'car_wash', 'bicycle_repair_station', 'hairdresser'
]

shopping_retail_points = points_of_interest_points[points_of_interest_points['amenity'].isin(shopping_amenities)]
shopping_retail_points.loc[shopping_retail_points['amenity'] == 'butchery', 'amenity'] = 'butcher'

points_of_interest_points = points_of_interest_points[~points_of_interest_points['amenity'].isin(shopping_amenities)]

In [679]:
shopping_retail_points.drop(['name:en', 'man_made', 'shop', 'tourism',
       'opening_hours', 'beds', 'rooms', 'addr:full', 'addr:housenumber',
       'addr:street', 'addr:city', 'source', 'name:sw'], axis = 1, inplace=True)

shopping_retail_points = extract_coordinates_from_geometry(shopping_retail_points)

Number of points processed: 1781
Number of missing coordinates: 0


/var/folders/b7/2dqsctv50sbgk3bg9gpz453c0000gn/T/ipykernel_22851/3776026523.py:15: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['longitude'] = gdf.geometry.centroid.x
/var/folders/b7/2dqsctv50sbgk3bg9gpz453c0000gn/T/ipykernel_22851/3776026523.py:16: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['latitude'] = gdf.geometry.centroid.y


In [ ]:
points_of_interest_points['amenity'].value_counts()

## Religious Buildings

In [694]:
religious_amenities = [
    'place_of_worship', 'church', 'mosque', 'monastery'
]

religious_buildings_points = points_of_interest_points[points_of_interest_points['amenity'].isin(religious_amenities)]

points_of_interest_points = points_of_interest_points[~points_of_interest_points['amenity'].isin(religious_amenities)]

In [ ]:
religious_buildings_points.drop(['name:en', 'man_made', 'shop', 'tourism',
       'opening_hours', 'beds', 'rooms', 'addr:full', 'addr:housenumber',
       'addr:street', 'addr:city', 'source', 'name:sw'], axis = 1, inplace=True)

religious_buildings_points = extract_coordinates_from_geometry(religious_buildings_points)

# Working With the Crimes Data CSV 

In [732]:
crimes_counties = pd.read_csv("../data/KEN_county_subcounty_crime_2022.csv")

In [ ]:
sub_counties = gpd.read_file('../data/ken_adm_iebc_20191031_shp/ken_admbnda_adm2_iebc_20191031.shp')

In [ ]:
# Ensure the sub-county names are consistent (e.g., case-insensitive comparison)
sub_counties['ADM2_EN'] = sub_counties['ADM2_EN'].str.strip().str.lower()
crimes_counties['sub-county'] = crimes_counties['sub-county'].str.strip().str.lower()

# Identify sub-counties in df2 that are not in df1
unmatched_sub_counties = crimes_counties[~crimes_counties['sub-county'].isin(sub_counties['ADM2_EN'])]

# Output the unmatched sub-counties
print("Sub-counties in df2 that do not match any in df1:")
print(unmatched_sub_counties['sub-county'].unique())


In [ ]:
def get_subcounty_from_coordinates(df, sub_counties):
    geometry = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]
    h3_gdf = gpd.GeoDataFrame(df, geometry=geometry, crs='EPSG:4326')

    # 2. Perform spatial join
    # This will add all columns from counties to your H3 data
    result = gpd.sjoin(h3_gdf, sub_counties, how='left', predicate='within')

    # 3. If you only want to keep the county name (assuming it's in 'ADM1_EN')
    # Keep only the original columns plus the county name
    original_columns = list(df.columns)
    df['sub_county'] = result['ADM2_EN']

    # # 4. Some hexagons are left unmatched so perform other operations to match them with a county
    unmatched_hexagons = df[df['sub_county'].isna()].copy()
    geometry = [Point(xy) for xy in zip(unmatched_hexagons['longitude'], unmatched_hexagons['latitude'])]
    h3_gdf = gpd.GeoDataFrame(unmatched_hexagons, geometry=geometry, crs='EPSG:4326')
    intersect_match = gpd.sjoin(h3_gdf, sub_counties, how='left', predicate='intersects')

    print(f"Number of unmatched hexagons = {len(unmatched_hexagons)}")
    print(f"Number of intersect matches = {len(intersect_match)}")
    
    def assign_nearest_county(point, counties_gdf):
        # Calculate distance to all counties
        distances = counties_gdf.geometry.distance(point.geometry)
        # Get the county with minimum distance
        nearest_county = counties_gdf.iloc[distances.argmin()]
        return nearest_county['ADM2_EN']

    # For any hexagons still unmatched after intersects, find nearest county
    still_unmatched = intersect_match[intersect_match['sub_county'].isna()]
    if len(still_unmatched) > 0:
        still_unmatched['nearest_sub_county'] = still_unmatched.apply(
            lambda x: assign_nearest_county(x, sub_counties), axis=1
        )

    df.loc[still_unmatched.index, 'sub_county'] = still_unmatched['nearest_sub_county']
    return df

In [ ]:
h3_with_poverty_sub_counties = get_subcounty_from_coordinates(h3_with_poverty.copy(), sub_counties)

In [765]:
h3_with_poverty_sub_counties['sub_county'] = h3_with_poverty_sub_counties['sub_county'].str.strip().str.lower()
crimes_counties['sub-county'] = crimes_counties['sub-county'].str.strip().str.lower()

# Add the 'high_crime_in_county' column with default value of 0
h3_with_poverty_sub_counties['high_crime_in_county'] = 0

# Update 'high_crime_in_county' to 1 for matches in df2
h3_with_poverty_sub_counties.loc[h3_with_poverty_sub_counties['sub_county'].isin(crimes_counties['sub-county']), 'high_crime_in_county'] = 1

# Waterways

In [772]:
waterways_lines = gpd.read_file('../data/ken_waterways_lines/ken_waterways_lines.gpkg')

In [773]:
waterways_lines.drop(['name', 'name:en', 'covered', 'width', 'depth', 'layer',
       'blockage', 'tunnel', 'natural', 'water', 'source', 'name:sw'], axis = 1, inplace=True)

In [ ]:
waterways_lines['waterway'].value_counts()

In [ ]:
print_df_info(waterways_lines, 'waterways_lines')

## Mapping

In [ ]:
def hexagons_dataframe_to_geojson(df_hex, file_output = None, column_name = "total_population"):
    """
    Produce the GeoJSON for a dataframe, constructing the geometry from the "hex_id" column
    and with a property matching the one in column_name
    """    
    list_features = []
    
    for i,row in df_hex.iterrows():
        try:
            geometry_for_row = { "type" : "Polygon", "coordinates": [h3.h3_to_geo_boundary(h=row["h3"],geo_json=True)]}
            feature = Feature(geometry = geometry_for_row , id=row["h3"], properties = {column_name : row[column_name]})
            list_features.append(feature)
        except:
            print("An exception occurred for hex " + row["h3"]) 

    feat_collection = FeatureCollection(list_features)
    geojson_result = json.dumps(feat_collection)
    return geojson_result

In [ ]:
def choropleth_map(df_aggreg, border_color = 'black', fill_opacity = 0.7, initial_map = None, with_legend = False,
                   kind = "linear"):   
    #colormap
    min_value = df_aggreg["total_population"].min()
    max_value = df_aggreg["total_population"].max()
    m = round ((min_value + max_value ) / 2 , 0)
    
    #take resolution from the first row
    res = 8
    
    if initial_map is None:
        initial_map = Map(location= [-1.286389, 36.817223], zoom_start=6, tiles="cartodbpositron", 
                attr= '© <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors © <a href="http://cartodb.com/attributions#basemaps">CartoDB</a>' 
            )
        

    #the colormap 
    #color names accepted https://github.com/python-visualization/branca/blob/master/branca/_cnames.json
    if kind == "linear":
        custom_cm = cm.LinearColormap(['green','yellow','red'], vmin=min_value, vmax=max_value)
    elif kind == "outlier":
        #for outliers, values would be -11,0,1
        custom_cm = cm.LinearColormap(['blue','white','red'], vmin=min_value, vmax=max_value)
    elif kind == "filled_nulls":
        custom_cm = cm.LinearColormap(['sienna','green','yellow','red'], 
                                      index=[0,min_value,m,max_value],vmin=min_value,vmax=max_value)
   

    #create geojson data from dataframe
    geojson_data = hexagons_dataframe_to_geojson(df_hex = df_aggreg)
    
    #plot on map
    name_layer = "Choropleth " + str(res)
    if kind != "linear":
        name_layer = name_layer + kind
        
    GeoJson(
        geojson_data,
        style_function = lambda feature: {
            'fillColor': custom_cm(feature['properties']['total_population']),
            'color': border_color,
            'weight': 1,
            'Highlight': True,
            'fillOpacity': fill_opacity 
        }, 
        name = name_layer
    ).add_to(initial_map)

    #add legend (not recommended if multiple layers)
    if with_legend == True:
        custom_cm.add_to(initial_map)   
    
    
    return initial_map

In [ ]:
hexmap = choropleth_map(df_aggreg = financial_services[financial_services['name'].isna()], with_legend = True, kind = "filled_nulls") 